In [ ]:
from ultralytics import YOLO
from wandb.integration.ultralytics import add_wandb_callback
import os  
import wandb
import yaml
import warnings 
warnings.filterwarnings(action='ignore', category=UserWarning)

### [YOLO Train Settings](https://docs.ultralytics.com/modes/train/#resuming-interrupted-trainings:~:text=a%20training%20run.-,Train%20Settings,-The%20training%20settings)

In [ ]:
# Initialize WandB
project="human_parsing"

wandb.init(project=project)

# Load a model
pretrained = "weights/yolov8n-seg.pt"
pretrained = "/home/jordan/SEGMENT/human_parsing/train17/weights/last.pt"


model = YOLO(
    pretrained, task="segment"
)  # Load a pretrained model (recommended for training)

# Load labels from YAML configuration file
yaml_file = "configs/fashion_people_detection.yml"
with open(yaml_file, 'r') as f:
    config = yaml.safe_load(f)
    class_labels = config['names']  # Adjust the key based on your YAML structure

# Add WandB callback
add_wandb_callback(model)

# Set the batch size and number of workers
bs = 36
workers = os.cpu_count()

# Train the model
results = model.train(
    project=project,
    data=yaml_file,
    epochs=2,
    imgsz=1024,
    batch=bs,
    workers=workers,
    augment=True,
    device=[0],
    patience=20,
    save_period=5,
    cache=True,
)

# Finish the W&B run
wandb.finish()